<a href="https://colab.research.google.com/github/aguiarsm/WebScraping_TripAdv/blob/main/1_Explorando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerenciando pacotes

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
#!apt upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.

In [ ]:
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# Extraindo links dos Hoteis

Aqui, vou raspar os dados da página que contém os 25 melhores hoteis do Brasil

## Filtrando um caso

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

#link da lista 
url = 'https://www.tripadvisor.com.br/TravelersChoice-Hotels-cTop-g294280'
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
all_hotel = soup.find_all("div", attrs={"class", "winnerName"})

In [ ]:
len(all_hotel)

25

In [ ]:
all_hotel[0:3]

[<div class="winnerName">
 <div class="mainName extra">
 <a href="/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink', 'Top-294280', 1, this.href);" target="_blank">1. Hotel Colline de France</a>
 </div>
 <div class="smaller">
 <a href="/Tourism-g303536-Gramado_State_of_Rio_Grande_do_Sul-Vacations.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink2', 'Top-294280', 1, this.href);" target="_blank">Gramado, Brasil</a>
 </div>
 </div>, <div class="winnerName">
 <div class="mainName extra">
 <a href="/Hotel_Review-g303536-d7255333-Reviews-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink', 'Top-294280', 2, this.href);" target="_blank">2. Hotel Valle D’Incanto</a>
 </div>
 <div class="smaller">
 <a href="/Tourism-g303536-Gramado_State_of_Rio_Grande_do_Sul-Vacations.html" onclick="ta.setEvtCookie('PhotoVie

In [ ]:
all_hotel[0].find_all("a")

[<a href="/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink', 'Top-294280', 1, this.href);" target="_blank">1. Hotel Colline de France</a>,
 <a href="/Tourism-g303536-Gramado_State_of_Rio_Grande_do_Sul-Vacations.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink2', 'Top-294280', 1, this.href);" target="_blank">Gramado, Brasil</a>]

In [ ]:
all_hotel[0].find_all("a")[0]['href'] #OK! Me dá o link

'/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html'

In [ ]:
all_hotel[0].find_all("a")[0]

<a href="/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink', 'Top-294280', 1, this.href);" target="_blank">1. Hotel Colline de France</a>

In [ ]:
all_hotel[0].find_all("a")[1]

<a href="/Tourism-g303536-Gramado_State_of_Rio_Grande_do_Sul-Vacations.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink2', 'Top-294280', 1, this.href);" target="_blank">Gramado, Brasil</a>

In [ ]:
#Nome do Hotel
re.search(r'target="_blank">.?.?.?(.*)<\/a>', str(all_hotel[0].find_all("a")[0])).group(1) 

'Hotel Colline de France'

In [ ]:
# Cidade do Hotel
re.search(f'target="_blank">(.*), Brasil<\/a>', str(all_hotel[0].find_all("a")[1])).group(1)

'Gramado'

## Generalizando e gerando dataframe

In [ ]:
data = []
for pos, prov in enumerate(all_hotel):
    endereco = prov.find_all("a")[0]['href']
    cidade = re.search(f'target="_blank">(.*), Brasil<\/a>', str(prov.find_all("a")[1])).group(1)
    nome = re.search(r'target="_blank">[0-9\. ]*(.*)<\/a>', str(prov.find_all("a")[0])).group(1)
    data.append([nome, cidade, "https://www.tripadvisor.com.br"+endereco])

In [ ]:
df_lista = pd.DataFrame(data, columns = ['nome_hotel', 'cidade', 'link'])

In [ ]:
df_lista #DataFrame com o link da página inicial de cada Hotel

,nome_hotel,cidade,link
0,Hotel Colline de France,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
1,Hotel Valle D’Incanto,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
2,Hotel Ritta Höppner,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
3,Hotel Refugio da Montanha,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
4,Hotel Estalagem St Hubertus,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
5,Hotel Le Renard,Campos do Jordão,https://www.tripadvisor.com.br/Hotel_Review-g3...
6,Carballo Hotel &amp; Spa,Campos do Jordão,https://www.tripadvisor.com.br/Hotel_Review-g3...
7,Porto Pacuíba Hotel,Ilhabela,https://www.tripadvisor.com.br/Hotel_Review-g6...
8,Salinas Maragogi All Inclusive Resort,Maragogi,https://www.tripadvisor.com.br/Hotel_Review-g6...
9,Casas Brancas Boutique Hotel &amp; Spa,Armação dos Búzios,https://www.tripadvisor.com.br/Hotel_Review-g3...


In [ ]:
links_hoteis = df_lista['link'].to_list()

In [ ]:
links_hoteis

['https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d778984-Reviews-Hotel_Ritta_Hoppner-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d12398420-Reviews-Hotel_Refugio_da_Montanha-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d1837047-Reviews-Hotel_Estalagem_St_Hubertus-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303607-d1958657-Reviews-Hotel_Le_Renard-Campos_Do_Jordao_State_of_Sao_Paulo.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303607-d6888786-Reviews-Carballo_Hotel_Spa-Campos_Do_Jordao_State_of_Sao_Paulo.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g6091

# Extraindo avaliações para um único hotel.

In [ ]:
links_hoteis[0]

'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html'

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

#link da lista 
driver.get(links_hoteis[0])

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
soup

<html lang="pt-BR" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/><link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/><meta content="#34e0a1" name="theme-color"/><meta content="telephone=no" name="format-detection"/><script type="text/javascript">window.taRollupsAreAsync = true;</script><link crossorigin="" href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans.css?v1.002" rel="stylesheet"/><link as="fetch" crossorigin="anonymous" href="/static/decodeKey.txt" rel="preload"/><title>HOTEL COLLINE DE FRANCE (GRAMADO): 3.225 fotos, comparação de preços e 2.586 avaliações - Tripadvisor</title><meta content="TripAdvisor" property="al:ios:app_name"/><meta content="284876795" property="al:ios:app_store_id"/><meta content="284876795" na

## Conseguindo link de cada página

No caso, dentro de um mesmo hotel. As avaliações aparecem somente com 5 por página.

In [ ]:
prox_page = soup.find_all("a", attrs={"class", "ui_button nav next primary "})

In [ ]:
prox_page

[<a class="ui_button nav next primary " href="/Hotel_Review-g303536-d13871137-Reviews-or5-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html">Próximas</a>]

As próximas páginas são rotuladas pelo "orXXX". Então fica simples gerar os próximos links. 

In [ ]:
prox_page[0]['href']

'/Hotel_Review-g303536-d13871137-Reviews-or5-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html'

In [ ]:
link_split = prox_page[0]['href'].split('-or5-')

In [ ]:
for num_coment_inf in range(0, 21, 5):
    if num_coment_inf == 0:
        link = "https://www.tripadvisor.com.br" + link_split[0] + '-' + link_split[1]
    else:
        link = "https://www.tripadvisor.com.br" + link_split[0] + '-' + 'or' + str(num_coment_inf) + '-' + link_split[1]
    print(link)

https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html
https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or5-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html
https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or10-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html
https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or15-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html
https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or20-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html


## Raspando dados de uma página


In [ ]:
link #Página qualquer para exploração

'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or20-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html'

In [ ]:
#Esta célula gera o soup da página. Resta apenas filtrar as informações e guardar. 

driver = webdriver.Chrome('chromedriver',options=chrome_options)

#link da lista 
driver.get(link)

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

### Notas

In [ ]:
nota_soup = soup.find_all("div", attrs={"class", "Hlmiy F1"})

In [ ]:
nota_soup

[<div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>,
 <div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>,
 <div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>,
 <div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>,
 <div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>]

In [ ]:
nota = int(re.findall(r'rating bubble_(..)', str(nota_soup))[0]) // 10
nota

5

In [ ]:
notas = []
for prov in nota_soup:
    nota = int(re.findall(r'rating bubble_(..)', str(prov))[0]) // 10
    notas.append(nota)
notas

[5, 5, 5, 5, 5]

### Texto da avaliação

In [ ]:
coment_soup = soup.find_all("q", attrs={"class", "QewHA H4 _a"})

In [ ]:
coment_soup

[<q class="QewHA H4 _a"><span>O hotel mais top que já fiquei em minha vida! Não foi apenas uma estadia! Foi uma experiência fantástica! O hotel tem tudo de ótimo! O atendimento é de outro mundo, a comida fantástica, em cada detalhe tem algo especial, sem dúvidas que eu vou voltar</span></q>,
 <q class="QewHA H4 _a"><span>Hotel mais incrível, vale muito a pena, a recepção dos funcionários, tudo perfeito, eu e meu esposo nos sentimos muito bem recebidos e tratados com tanta gentileza e educação, que não conseguimos parar de falar sobre o hotel, saímos de lá já querendo voltar! Não tenho nada para sugerir, porque tudo foi tão perfeito, superou todas minhas expectativas, que já eram altas rsrs! Até breve Colline de France ♥️</span></q>,
 <q class="QewHA H4 _a"><span>Lugar encantador, atendimento impecável.  Um chá acompanhado dos melhores e mais delicados doces. Ansiosa para me hospedar e desfrutar de toda a maravilha que é este hotel 👏 Detalhes para se apaixonar!</span></q>,
 <q class="Qe

In [ ]:
coment = re.findall(r'<span>(.*)<\/span>', str(coment_soup[0]))
coment

['O hotel mais top que já fiquei em minha vida! Não foi apenas uma estadia! Foi uma experiência fantástica! O hotel tem tudo de ótimo! O atendimento é de outro mundo, a comida fantástica, em cada detalhe tem algo especial, sem dúvidas que eu vou voltar']

In [ ]:
coments = []
for prov in coment_soup:
    coment = re.findall(r'<span>(.*)<\/span>', str(prov))
    coments.append(coment[0])
coments

['O hotel mais top que já fiquei em minha vida! Não foi apenas uma estadia! Foi uma experiência fantástica! O hotel tem tudo de ótimo! O atendimento é de outro mundo, a comida fantástica, em cada detalhe tem algo especial, sem dúvidas que eu vou voltar',
 'Hotel mais incrível, vale muito a pena, a recepção dos funcionários, tudo perfeito, eu e meu esposo nos sentimos muito bem recebidos e tratados com tanta gentileza e educação, que não conseguimos parar de falar sobre o hotel, saímos de lá já querendo voltar! Não tenho nada para sugerir, porque tudo foi tão perfeito, superou todas minhas expectativas, que já eram altas rsrs! Até breve Colline de France ♥️',
 'Lugar encantador, atendimento impecável.  Um chá acompanhado dos melhores e mais delicados doces. Ansiosa para me hospedar e desfrutar de toda a maravilha que é este hotel 👏 Detalhes para se apaixonar!',
 'Excelente hospedagem. Nos proporcionaram férias inesquecíveis. Com certeza, voltaremos. Acomodações de primeira, serviços de 

### Titulo da avaliação

In [ ]:
titulo_soup = soup.find_all("div", attrs={"class", "KgQgP MC _S b S6 H5 _a"})
titulo_soup

[<div class="KgQgP MC _S b S6 H5 _a" data-test-target="review-title" dir="ltr"><a class="Qwuub" href="/ShowUserReviews-g303536-d13871137-r848631690-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html"><span><span>Fantástico</span></span></a></div>,
 <div class="KgQgP MC _S b S6 H5 _a" data-test-target="review-title" dir="ltr"><a class="Qwuub" href="/ShowUserReviews-g303536-d13871137-r848630083-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html"><span><span>A melhor experiência</span></span></a></div>,
 <div class="KgQgP MC _S b S6 H5 _a" data-test-target="review-title" dir="ltr"><a class="Qwuub" href="/ShowUserReviews-g303536-d13871137-r848622934-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html"><span><span>Conhecendo o melhor hotel do mundo</span></span></a></div>,
 <div class="KgQgP MC _S b S6 H5 _a" data-test-target="review-title" dir="ltr"><a class="Qwuub" href="/ShowUserReviews-g303536-d13871137-r848621957-Hotel_Colline_de_France-Gramado_Sta

In [ ]:
titulo = re.findall(r'<span><span>(.*)<\/span><\/span>', str(titulo_soup[0]))
titulo

['Fantástico']

In [ ]:
titulos = []
for prov in titulo_soup:
    titulo = re.findall(r'<span><span>(.*)<\/span><\/span>', str(prov))
    titulos.append(titulo[0])
titulos

['Fantástico',
 'A melhor experiência',
 'Conhecendo o melhor hotel do mundo',
 'Férias inesqueciveis',
 'Melhor impossível!!']

### Data avaliação

In [ ]:
data_aval_soup = soup.find_all("div", attrs={"class", "cRVSd"})
data_aval_soup

[<div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/diegoaV5502EV">Diego A</a> escreveu uma avaliação 18 de jul.</span></div>,
 <div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/eduardatossan">Eduarda S</a> escreveu uma avaliação 18 de jul.</span></div>,
 <div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/EduardaT77">Eduarda T</a> escreveu uma avaliação 18 de jul.</span></div>,
 <div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/Tarcisio_ime_02">Tarcísio Freitas</a> escreveu uma avaliação 18 de jul.</span></div>,
 <div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/marcioc309">Marcio C</a> escreveu uma avaliação 18 de jul.</span></div>]

In [ ]:
data_aval = re.findall(r'escreveu uma avaliação (.*)<\/span><\/div>', str(data_aval_soup[0]))
data_aval

['18 de jul.']

In [ ]:
data_avals = []
for prov in data_aval_soup:
    data_aval = re.findall(r'escreveu uma avaliação (.*)<\/span><\/div>', str(prov))
    data_avals.append(data_aval[0])
data_avals

['18 de jul.', '18 de jul.', '18 de jul.', '18 de jul.', '18 de jul.']

### Data da estadia

In [ ]:
data_esta_soup = soup.find_all("span", attrs={"class", "teHYY _R Me S4 H3"})
data_esta_soup

[<span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> julho de 2022</span>,
 <span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> julho de 2022</span>,
 <span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> julho de 2022</span>,
 <span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> dezembro de 2021</span>,
 <span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> julho de 2022</span>]

In [ ]:
data_esta = re.findall(r'<\/span> (.*)<\/span>', str(data_esta_soup[0]))
data_esta

['julho de 2022']

In [ ]:
data_estas = []
for prov in data_esta_soup:
    data_esta = re.findall(r'<\/span> (.*)<\/span>', str(prov))
    data_estas.append(data_esta[0])
data_estas

['julho de 2022',
 'julho de 2022',
 'julho de 2022',
 'dezembro de 2021',
 'julho de 2022']

### Resumindo: definindo uma função geradora

In [ ]:
def raspa_pagina(soup, tag_html, atributo, match_atributo, regex):
    
    
    data_soup = soup.find_all(tag_html, attrs={atributo, match_atributo})
    x  = []
    
    for prov in (data_soup):
        _ = re.findall(regex, str(prov))
        
        if _ != []:
            if regex == 'rating bubble_(..)':
                x.append(int(_[0]) // 10)
            else:
                x.append(_[0])
        else: 
            x.append("ERRO!!!")

    return x


In [ ]:
raspa_pagina(soup, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS

[5, 5, 5, 5, 5]

In [ ]:
raspa_pagina(soup, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO

['O hotel mais top que já fiquei em minha vida! Não foi apenas uma estadia! Foi uma experiência fantástica! O hotel tem tudo de ótimo! O atendimento é de outro mundo, a comida fantástica, em cada detalhe tem algo especial, sem dúvidas que eu vou voltar',
 'Hotel mais incrível, vale muito a pena, a recepção dos funcionários, tudo perfeito, eu e meu esposo nos sentimos muito bem recebidos e tratados com tanta gentileza e educação, que não conseguimos parar de falar sobre o hotel, saímos de lá já querendo voltar! Não tenho nada para sugerir, porque tudo foi tão perfeito, superou todas minhas expectativas, que já eram altas rsrs! Até breve Colline de France ♥️',
 'Lugar encantador, atendimento impecável.  Um chá acompanhado dos melhores e mais delicados doces. Ansiosa para me hospedar e desfrutar de toda a maravilha que é este hotel 👏 Detalhes para se apaixonar!',
 'Excelente hospedagem. Nos proporcionaram férias inesquecíveis. Com certeza, voltaremos. Acomodações de primeira, serviços de 

In [ ]:
raspa_pagina(soup, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO

['Fantástico',
 'A melhor experiência',
 'Conhecendo o melhor hotel do mundo',
 'Férias inesqueciveis',
 'Melhor impossível!!']

In [ ]:
raspa_pagina(soup, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA


['18 de jul.', '18 de jul.', '18 de jul.', '18 de jul.', '18 de jul.']

## Validando

In [ ]:
links_hoteis #OK

['https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d778984-Reviews-Hotel_Ritta_Hoppner-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d12398420-Reviews-Hotel_Refugio_da_Montanha-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d1837047-Reviews-Hotel_Estalagem_St_Hubertus-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303607-d1958657-Reviews-Hotel_Le_Renard-Campos_Do_Jordao_State_of_Sao_Paulo.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303607-d6888786-Reviews-Carballo_Hotel_Spa-Campos_Do_Jordao_State_of_Sao_Paulo.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g6091

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

#link da lista 
driver.get(links_hoteis[0])

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
raspa_pagina(soup, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS

[5, 5, 5, 5, 5]

In [ ]:
raspa_pagina(soup, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO

['Excelente! Local lindíssimo, com charme e requinte por onde você anda. Sem falar nos aromas em todos os ambientes, marca registrada do Colline. Mas o ponto mais forte, são as pessoas: atendimento fantástico e acolhedor. Foram incansáveis para atender um pedido meu de último hora: e saiu fantástico! Pessoas como o Luca, Fabiano e Greice fazem o seu dia começar muito bem. Com certeza, voltaremos! Até breve, Colline!',
 'Espetacular o atendimento, as acomodações, a decoração, sem  Café da manhã, o piano!! Sem palavras para descrever a nossa satisfação. É muito mais do que tudo que ouvimos falar a respeito do hotel. Faz jus em figurar entre os melhores do mundo. Valeu a pena. Com certeza retornaremos.',
 'Melhor experiência que já tive em uma viagem!!!!!!⭐️⭐️⭐️⭐️⭐️  Tudo simplesmente perfeito! Pensam em cada detalhe para proporcionar a melhor experiência aos hóspedes! Funcionários educados, atenciosos e cordiais! Sempre sorrindo e dispostos! Educação e cordialidade ímpares! Limpeza, comi

In [ ]:
raspa_pagina(soup, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO

['Final de Semana Romântico',
 'Fantástico o atendimento as acomodações e tudo mais que eles fazem para agradar os hóspedes. Nada igual até então!',
 'Não à toa é o MELHOR HOTEL DO MUNDO!',
 'O melhor hotel de Gramado ',
 'Experiência fantástica']

In [ ]:
raspa_pagina(soup, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA


['Ontem', 'Ontem', 'Ontem', 'Ontem', 'Ontem']

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

#link da lista (Outro Hotel)
driver.get(links_hoteis[1])

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
_nota = raspa_pagina(soup, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS

In [ ]:
_texto = raspa_pagina(soup, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO

In [ ]:
_titulo = raspa_pagina(soup, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO

In [ ]:
_data = raspa_pagina(soup, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA


In [ ]:
teste = pd.DataFrame(np.transpose(np.array([_nota, _titulo, _texto, _data])))
teste

,0,1,2,3
0,5,Melhor hotel que já visitei,Primeira vez que paro para avaliar um hotel pq...,Ontem
1,5,Sempre uma excelente experiência!,Retornamos ao hotel devido a excelente experiê...,fev. de 2022
2,5,Hospedagem perfeita,Fomos em família e somos só elogios. Todos os ...,24 de jul.
3,5,Excelente,Atendimento e acomodações impecáveis! Vale a p...,24 de jul.
4,5,Comemorando aniversário de casamento,Tudo impecável Fomos surpreendidos positivame...,24 de jul.


## Extraindo várias páginas

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

#link da lista 
driver.get(links_hoteis[1])

soup_hotel = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
prox_page = soup_hotel.find_all("a", attrs={"class", "ui_button nav next primary "})
link_split = prox_page[0]['href'].split('-or5-')

link_hotel_paginas = []

for num_coment_inf in range(0, 21, 5):
    if num_coment_inf == 0:
        link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + link_split[1])
    else:
        link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + 'or' + str(num_coment_inf) + '-' + link_split[1])

        

df_hotel = pd.DataFrame(columns = ['nota', 'titulo', 'texto', 'data', 'ano'])
link_hotel_paginas

['https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-or5-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-or10-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-or15-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-or20-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html']

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

#link da lista 
driver.get(link_hotel_paginas[1])

soup_hotel_paginas = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
_nota = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS

In [ ]:
_texto = raspa_pagina(soup_hotel_paginas, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO

In [ ]:
_titulo = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO

In [ ]:
_data = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA
_data

['24 de jul.', '24 de jul.', '22 de jul.', '21 de jul.', '21 de jul.']

In [ ]:
_ano = []
_este_ano = ['Hoje', 'Ontem', 'jan.', 'fev.', 'mar.', 'abr.', 'mai.', 'jun.', 'jul.', 'ago.', 'set.', 'out.', 'nov.', 'dez.']
for prov in _data:
    if prov.split()[-1] in _este_ano:
        _ano.append(int(2022))
    else:
        _ano.append(int(prov.split()[-1]))
_ano

[2022, 2022, 2022, 2022, 2022]

In [ ]:
_ = pd.DataFrame(np.transpose(np.array([_nota, _titulo, _texto, _data, _ano])), columns = ['nota', 'titulo', 'texto', 'data', 'ano'])
df_hotel = pd.concat([df_hotel, _], ignore_index=True)

In [ ]:
df_hotel

,nota,titulo,texto,data,ano
0,5,O hotel mais perfeito de Gramado,"Nossa experiência foi simplesmente perfeita, o...",24 de jul.,2022
1,5,Experiência maravilhosa!!,Hotel muito charmoso e atendimento excelente! ...,24 de jul.,2022
2,5,Hotel encantador,Uma experiência única que em uma diária já se ...,22 de jul.,2022
3,5,Experiência espetacular! Equipe super alinhada...,"Hotel limpo, novo, tudo cheiroso! Tivemos uma...",21 de jul.,2022
4,5,"Atendimento encantador, experiência inigualável.",Hospedagem maravilhosa para quem busca excelên...,21 de jul.,2022


## Testando para primeiro Hotel

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

#link da lista 
driver.get(links_hoteis[0])

soup_hotel = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
prox_page = soup_hotel.find_all("a", attrs={"class", "ui_button nav next primary "})
link_split = prox_page[0]['href'].split('-or5-')

link_hotel_paginas = []
limite = 3 #Quantidade de páginas a serem extraídas
for num_coment_inf in range(0, limite * 5, 5):
    if num_coment_inf == 0:
        link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + link_split[1])
    else:
        link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + 'or' + str(num_coment_inf) + '-' + link_split[1])

        

df_hotel = pd.DataFrame(columns = ['nota', 'titulo', 'texto', 'data', 'ano'])
link_hotel_paginas

['https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or5-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or10-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html']

In [ ]:
df_hotel

,nota,titulo,texto,data,ano


In [ ]:
_ano = [2022, 2022, 2022, 2022, 2022]
pagina = 0
ano_limite = 2021 #Avaliações à partir de

while _ano[-1] >= 2021 and pagina < limite:
    driver = webdriver.Chrome('chromedriver',options=chrome_options)

    #link da lista 
    driver.get(link_hotel_paginas[pagina])

    soup_hotel_paginas = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    
    _nota = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS
    _texto = raspa_pagina(soup_hotel_paginas, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO
    _titulo = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO
    _data = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA
    _ano = []
    

    for prov in _data:
        if prov.split()[-1] in _este_ano:
            _ano.append(int(2022))
        else:
            _ano.append(int(prov.split()[-1]))    
    
    _ = pd.DataFrame(np.transpose(np.array([_nota, _titulo, _texto, _data, _ano])), columns = ['nota', 'titulo', 'texto', 'data', 'ano'])
    df_hotel = pd.concat([df_hotel, _], ignore_index=True)
    
    print(_data[-1])
    pagina += 1

Ontem
22 de jul.
19 de jul.


In [ ]:
df_hotel

,nota,titulo,texto,data,ano
0,5,Final de Semana Romântico,"Excelente! Local lindíssimo, com charme e requ...",Ontem,2022
1,5,Fantástico o atendimento as acomodações e tudo...,"Espetacular o atendimento, as acomodações, a d...",Ontem,2022
2,5,Não à toa é o MELHOR HOTEL DO MUNDO!,Melhor experiência que já tive em uma viagem!!...,Ontem,2022
3,5,O melhor hotel de Gramado,"Simplesmente encantador, tudo impecável! Aten...",Ontem,2022
4,5,Experiência fantástica,Com certeza esse hotel merece o título o qual ...,Ontem,2022
5,5,Passeio,"Experiência incrível, desde que cheguei já sen...",Ontem,2022
6,5,Espetáculo!,Retornamos ao Colline de France. A cada visita...,24 de jul.,2022
7,5,25 anos de casados,Excelente!!! Melhor hotel da vidaaa!! Adoramo...,22 de jul.,2022
8,5,Melhor hotel do mundo.,"Tudo no hotel é maravilhoso, o atendimento é i...",22 de jul.,2022
9,5,25 anos de casamento,Ficamos muito satisfeito com o Hotel e com o a...,22 de jul.,2022


OK!!! Agora resta generalizar